In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbn
import missingno as msno

from davood_ml_functions import *
from warnings import filterwarnings
filterwarnings("ignore")

def arabic_to_int(s) :
    arabic_digits = {"٠":0 , "١":1 , "٢":2 , "٣":3 , "٤":4 , "٥":5 , "٦":6 , "٧":7 , "٨":8 , "٩":9}
    return int("".join(str(arabic_digits.get(ch , ch)) for ch in s))
def time_to_index(series) :
    result = ((series.dt.year - 2020) * 4 + ((series.dt.month - 1) // 3 + 1))
    result[series.dt.year < 2020] = 0
    return result

In [22]:
df = pd.read_csv('Divar.csv')
print("Number of Rows =" , df.shape[0])
print("Number of Columns =" , df.shape[1])

Number of Rows = 1000000
Number of Columns = 61


In [23]:
# categories
necessary_options = ["has_water" , "has_electricity" , "has_gas"]
lux_options = ["has_barbecue" , "has_pool" , "has_jacuzzi" , "has_sauna" , "has_security_guard"]
normal_options = ["has_balcony" , "has_elevator" , "has_warehouse" , "has_parking" ,
"has_warm_water_provider" , "has_heating_system" , "is_rebuilt" , "has_cooling_system" , 
"has_restroom"]

for_sale = ["residential-sell" , "commercial-sell"]
for_rent = ["residential-rent" , "commercial-rent" , "temporary-rent"]

residential = ["residential-sell" , "residential-rent" , "temporary-rent"]
commercial  = ["commercial-rent" , "commercial-sell"]

rent_related = ["rent_value" , "rent_type" , "rent_credit_transform" , "credit_value" ,
                "transformable_credit" , "transformed_credit" , "transformable_rent" , "transformed_rent"]
price_related = ["price_value" , "transformable_price"]

In [24]:
# column 00
# df = df.drop("cat2_slug" , axis = 1)
df = df.dropna(subset = ["cat2_slug"])
df = df.loc[df["cat2_slug"] != "real-estate-services"]
# column 01
df = df.dropna(subset = ["cat3_slug"])
# column 02
df = df.dropna(subset = ["city_slug"])
# column 03
# df = df.drop("neighborhood_slug" , axis = 1)

# column 04
df = df.dropna(subset = ["created_at_month"])
df["created_at_month"] = pd.to_datetime(df["created_at_month"])
df["created_at_month"] = time_to_index(df["created_at_month"])

# column 05
df["user_type"] = df["user_type"].eq("مشاور املاک")
# column 06
#df = df.drop("description" , axis = 1)
# column 07
#df = df.drop("title" , axis = 1)
# column 08
df["rent_mode"] = df["rent_mode"].eq("مقطوع")
# column 10
df = df.drop("rent_to_single" , axis = 1)
# column 12
df["price_mode"] = df["price_mode"].eq("مقطوع")
# column 14
df["credit_mode"] = df["credit_mode"].eq("مقطوع")
# column 22
df = df.drop("land_size" , axis = 1)

# column 23
import re
import numpy as np

def fa_to_en_digits(s):
    if pd.isna(s):
        return s
    persian = "۰۱۲۳۴۵۶۷۸۹"
    english = "0123456789"
    return s.translate(str.maketrans(persian, english))

def extract_building_from_title(text):
    if pd.isna(text):
        return np.nan
    
    text = fa_to_en_digits(text)
    
    matches = re.findall(r'(\d+)\s*(?:متر(?:ی|مربع)?|m)', text, flags=re.IGNORECASE)
    if not matches:
        return np.nan
    
    nums = [int(m) for m in matches]
    
    return nums[0]

df["building_size_from_title"] = df["title"].apply(extract_building_from_title)

df["building_size"] = df["building_size"].fillna(df["building_size_from_title"])
df = df.drop(['building_size_from_title'] , axis = 1)


# column 24
df["deed_type"] = df["deed_type"].isin(["written_agreement" , "multi_page" , "other"])
# column 25
df["has_business_deed"] = df["has_business_deed"].isin(["True" , True])
# column  26
df["floor"] = df["floor"].replace("30+" , 30).astype(float)
# column 27
df = df.dropna(subset = ["rooms_count"])
df["rooms_count"] = df["rooms_count"].replace({"دو" : 2 , "یک" : 1 , "سه" : 3 , 
                           "بدون اتاق" : 0 , "چهار" : 4 , "پنج یا بیشتر" : 5})

# column 28
df["total_floors_count"] = df["total_floors_count"].replace("30+" , 30)
df["total_floors_count"] = df["total_floors_count"].replace("unselect" , np.nan)
df.loc[df["cat3_slug"].isin(["house-villa-sell" , "house-villa-rent" , "villa"]) , "total_floors_count"] = 0
df["total_floors_count"] = df["total_floors_count"].astype(float)
df["total_floors_count"] = np.where(df["total_floors_count"] > 10 , 2 , np.where(df["total_floors_count"].between(5 , 10) , 1 , 0))

# column 29
df = df.drop("unit_per_floor" , axis = 1)
# column 30
df["has_balcony"] = df["has_balcony"].isin([True , "true"])
# column 31
df["has_elevator"] = df["has_elevator"].eq(True)
# column 32
df["has_warehouse"] = df["has_warehouse"].eq(True)
# column 33
df["has_parking"] = df["has_parking"].eq(True)

# column 34
df["construction_year"] = df["construction_year"].replace("قبل از ۱۳۷۰" , "۱۳۷۰")
df["construction_year"] = df["construction_year"].fillna("۰")
df["construction_year"] = df["construction_year"].apply(arabic_to_int)
df["construction_year"] = df["construction_year"].replace(0 , np.nan)
df = df.dropna(subset = ["construction_year"])
df["construction_year"] = df["construction_year"].astype(int)

# column 35
df["is_rebuilt"] = df["is_rebuilt"].eq(True)

# column 36,38,39
for necessary_option in necessary_options:
    df[necessary_option] = ~df[necessary_option].eq(False)
del necessary_option

# column 37
df["has_warm_water_provider"] = df["has_warm_water_provider"].eq("package")
# column 40
df["has_heating_system"] = df["has_heating_system"].eq("shoofaj")
# column 41
df["has_cooling_system"] = df["has_cooling_system"].isin(["air_conditioner" , "split"])
# column 42
df["has_restroom"] = df["has_restroom"].isin(["squat_seat" , "seat"])
# column 43
df["has_security_guard"] = df["has_security_guard"].isin([True])

# columns 44,46,47,48
for lux_option in lux_options:
    df[lux_option] = df[lux_option].eq(True)
del lux_option

# column 45
df = df.drop("building_direction" , axis = 1)
# column 49
df["floor_material"] = df["floor_material"].isin(["stone" , "carpet" , "wood_parquet" , "laminate_parquet"])
# column 50
df["property_type"] = ~df["property_type"].isna()
# column 51
df["regular_person_capacity"] = df["regular_person_capacity"] > 10
# column 52
df["extra_person_capacity"] = df["extra_person_capacity"].replace("30+" , 30).astype(float) > 5
# column 53
df = df.drop("cost_per_extra_person" , axis = 1)
# column 54
df = df.drop("rent_price_on_regular_days" , axis = 1)
# column 55
df = df.drop("rent_price_on_special_days" , axis = 1)
# column 56
df = df.drop("rent_price_at_weekends" , axis = 1)

# column 57
group_means = df.groupby("neighborhood_slug")["location_latitude"].transform("mean")
df["location_latitude"] = df["location_latitude"].fillna(group_means)
df = df.dropna(subset = ["location_latitude"])

# column 58
group_means = df.groupby("neighborhood_slug")["location_longitude"].transform("mean")
df["location_longitude"] = df["location_longitude"].fillna(group_means)
df = df.dropna(subset = ["location_longitude"])

# column 59
df = df.drop("location_radius" , axis = 1)

# add new columns 
df["lux_count"] = df.loc[: , lux_options].sum(axis = 1)
df["necessary_count"] = df.loc[: , necessary_options].sum(axis = 1)
df["normal_count"] = df.loc[:, normal_options].sum(axis = 1)

df["mode_rent"] = df["rent_mode"].astype(int) + df["credit_mode"].astype(int)
df["mode_sale"] = df["price_mode"].astype(int)
df["mode"] = np.where(df["cat2_slug"].isin(for_sale) , df["mode_sale"] , np.where(df["cat2_slug"].isin(for_rent) , df["mode_rent"] , np.nan))
df = df.drop(["mode_rent" , "rent_mode" , "credit_mode" , "mode_sale" , "price_mode"] , axis = 1)
df["mode"] = df["mode"].astype(int)

In [25]:
# The Y vector
df["for_sale"] = df["cat2_slug"].isin(for_sale)
df["Y"] = np.where(df["for_sale"] == True , df["price_value"] , df["transformable_credit"])
df = df.drop(price_related , axis = 1)
df = df.drop(rent_related , axis = 1)
df["Y"] = df["Y"].astype(float)
df = df.dropna(subset = ["Y"])

In [26]:
# ----- EXAMINE THIS PART VERY CAREFULLY ----- #

## rename some columns
# rename_dict = {"cat3_slug" : "cat" , "city_slug" : "city" , 
#                      "neighborhood_slug" : "neighborhood" , "construction_year" : "age" ,
#                     "location_latitude" : "lat" , "location_longitude" : "long"}
# df = df.rename(columns = rename_dict)


## drop the columns you do not need
# df = df.drop(lux_options , axis = 1)
# df = df.drop(normal_options , axis = 1)
# df = df.drop(necessary_options , axis = 1)
# df = df.drop("cat2_slug" , axis = 1)

## run if your model does not take null values
# containing_null = ["neighborhood" , "floor"]
# df = df.drop(containing_null , axis = 1)

## converts boolean columns to int
# df[df.select_dtypes(bool).columns] = df.select_dtypes(bool).astype(int)

## get dummies
# df = pd.get_dummies(df)

## split by sale and rent
# df_sale = df.loc[df["cat2_slug"].isin(for_sale)]
# df_sale = df_sale.dropna()
# df_sale["Y"] = (df_sale["Y"] - df_sale["Y"].mean()) / df_sale["Y"].std(ddof = 1) # normalize

# df_rent = df.loc[df["cat2_slug"].isin(for_rent)]
# df_rent = df_rent.dropna()
# df_rent["Y"] = (df_rent["Y"] - df_rent["Y"].mean()) / df_rent["Y"].std(ddof = 1) # normalize

## if you wanta  dataframe with no null values (300K rows)
# df = df.dropna()

# ----- EXAMINE THIS PART VERY CAREFULLY ----- #

In [27]:
print("Number of Rows =" , df.shape[0])
print("Number of Columns =" , df.shape[1])

Number of Rows = 660622
Number of Columns = 45


In [28]:
table = get_table_null_dtype(df)
# table[table["dtype"] == "object"]
table[table["null_percentage"] > 0]

,null_percentage,dtype
column_name,,
neighborhood_slug,41.26,object
floor,29.86,float64
title,0.01,object


In [29]:
table[table["dtype"] == "object"]

,null_percentage,dtype
column_name,,
neighborhood_slug,41.26,object
title,0.01,object
cat2_slug,0.00,object
city_slug,0.00,object
cat3_slug,0.00,object
description,0.00,object


In [30]:
get_table_null_dtype(df)

,null_percentage,dtype
column_name,,
neighborhood_slug,41.26,object
floor,29.86,float64
title,0.01,object
Unnamed: 0,0.00,int64
cat2_slug,0.00,object
city_slug,0.00,object
cat3_slug,0.00,object
user_type,0.00,bool
created_at_month,0.00,int32


In [31]:
df.to_csv('clean_da.csv')

In [43]:
# print("Number of Rows =" , df.shape[0])
# print("Number of Columns =" , df.shape[1])
# fig, ax = plt.subplots(figsize=(15, 6))
# msno.matrix(df.loc[:,df.columns[df.isnull().any()]] ,  ax=ax, fontsize=12, sparkline=True, color=(0.2, 0.4, 0.6))
# ax.set_title("Missing Data Matrix", fontsize=16, weight='bold')
# fig.savefig("missing_data_matrix.png" , dpi = 400)
# plt.show()